## Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

import folium
from folium.plugins import MarkerCluster

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,1,Bethel,US,41.3712,-73.4140,66.56,83,5,3.53,clear sky
1,2,North Bend,US,43.4065,-124.2243,61.92,100,100,1.97,overcast clouds
2,3,Lompoc,US,34.6391,-120.4579,73.56,69,75,12.66,broken clouds
3,4,Wabash,US,40.7978,-85.8205,73.87,79,2,7.76,clear sky
4,5,Kapaa,US,22.0752,-159.3190,83.19,69,61,5.01,broken clouds


In [3]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,1,Bethel,US,41.3712,-73.4140,66.56,83,5,3.53,clear sky
1,2,North Bend,US,43.4065,-124.2243,61.92,100,100,1.97,overcast clouds
2,3,Lompoc,US,34.6391,-120.4579,73.56,69,75,12.66,broken clouds
3,4,Wabash,US,40.7978,-85.8205,73.87,79,2,7.76,clear sky
4,5,Kapaa,US,22.0752,-159.3190,83.19,69,61,5.01,broken clouds


In [5]:
# Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                634
City                   634
Country                634
Lat                    634
Lng                    634
Max Temp               634
Humidity               634
Cloudiness             634
Wind Speed             634
Current Description    634
dtype: int64

In [6]:
# Create DataFrame to store hotel names along with city, country, max temp, and coordinates.
va_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create new columns.
va_df['Airport'] = ""
va_df["Hotel"] = ""
va_df['Restaurant'] = ""

va_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Airport,Hotel,Restaurant
0,Bethel,US,66.56,clear sky,41.3712,-73.4140,,,
1,North Bend,US,61.92,overcast clouds,43.4065,-124.2243,,,
2,Lompoc,US,73.56,broken clouds,34.6391,-120.4579,,,
3,Wabash,US,73.87,clear sky,40.7978,-85.8205,,,
4,Kapaa,US,83.19,broken clouds,22.0752,-159.3190,,,


In [7]:
# Set parameters to search for airport with 250000 meters or 100+ miles.
params = {"radius": 250000, "type": "airport", "key": g_key}

# Iterate through the DataFrame.
for index, row in va_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    airport = requests.get(base_url, params=params).json()
    
    # Grab the first airport from the results and store the name.
    try:
        va_df.loc[index, "Airport"] = airport["results"][0]["name"]
    except (IndexError):
        print("Airport not found. Skipping...")

In [8]:
# Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000, "type": "lodging", "key": g_key}

# Iterate through the DataFrame.
for index, row in va_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        va_df.loc[index, "Hotel"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [9]:
# Set parameters to search for restaurant with 5000 meters.
params = {"radius": 5000, "type": "restaurant", "key": g_key}

# Iterate through the DataFrame.
for index, row in va_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    restaurants = requests.get(base_url, params=params).json()
    
    # Grab the first restaurant from the results and store the name.
    try:
        va_df.loc[index, "Restaurant"] = restaurants["results"][0]["name"]
    except (IndexError):
        print("Restaurant not found. Skipping...")

Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...
Restaurant not found. Skipping...


In [10]:
# Drop the rows where there is no place available.
clean_df = va_df.loc[(va_df["Airport"] != "") & (va_df["Hotel"] != "") & (va_df["Restaurant"] != "")]
clean_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Airport,Hotel,Restaurant
0,Bethel,US,66.56,clear sky,41.3712,-73.4140,Westchester County Airport,Courtyard by Marriott Danbury,Bangkok
1,North Bend,US,61.92,overcast clouds,43.4065,-124.2243,Southwest Oregon Regional Airport (KOTH),Red Lion Hotel Coos Bay,Subway
2,Lompoc,US,73.56,broken clouds,34.6391,-120.4579,Santa Maria Public Airport District,"Holiday Inn Express Lompoc, an IHG Hotel",Subway
3,Wabash,US,73.87,clear sky,40.7978,-85.8205,Miller Strip Airport-0II3,"Holiday Inn Express & Suites Wabash, an IHG Hotel",Honeywell Center
4,Kapaa,US,83.19,broken clouds,22.0752,-159.3190,Lihue Airport,Sheraton Kauai Coconut Beach Resort,Smith Family Garden Luau


In [11]:
clean_df.shape

(611, 9)

In [12]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a csv
clean_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# Using the template add city name, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Airport</dt><dd>{Airport}</dd>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>Restaurant</dt><dd>{Restaurant}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
va_info = [info_box_template.format(**row) for index, row in clean_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

In [14]:
max_temp = clean_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=va_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))